# Content based pipe

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle
import joblib
import sys
import jdc

from sklearn.ensemble import RandomForestRegressor

%load_ext autoreload
%autoreload 2 

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-13rll4li because the default path (/home/pjavier/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [78]:
from resype.preprocessing import *
from resype.cb_model import *
from resype.evaluate import *

In [3]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [85]:
user_df

,userId,u_1,u_2,u_3,u_4,u_5,u_6,u_7,u_8,u_9,...,u_11,u_12,u_13,u_14,u_15,u_16,u_17,u_18,u_19,u_20
0,1,85,29,42,83,47,26,90,45,55,...,17,68,22,7,40,22,1,0,0,0
1,2,3,0,0,7,0,1,11,10,10,...,1,17,1,1,4,0,0,4,3,0
2,3,11,4,5,9,4,5,14,2,7,...,8,16,5,0,15,1,0,0,0,0
3,4,29,6,10,104,19,58,25,27,38,...,4,120,7,10,12,16,4,1,2,0
4,5,8,6,9,15,7,11,9,12,9,...,1,25,3,2,2,5,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606,147,42,49,421,97,355,151,133,199,...,52,698,65,17,79,44,8,16,5,0
606,607,45,6,19,55,21,29,72,27,61,...,35,82,6,2,36,5,0,1,0,0
607,608,181,55,88,355,111,106,277,146,259,...,97,280,19,11,167,33,4,12,6,0
608,609,10,1,2,7,1,5,11,6,14,...,2,19,4,1,5,0,0,1,2,0


In [86]:
item_df

,movieId,i_1,i_2,i_3,i_4,i_5,i_6,i_7,i_8,i_9,...,i_291,i_292,i_293,i_294,i_295,i_296,i_297,i_298,i_299,i_300
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.513025,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9503,160341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.313289,0.0
9504,160527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
9505,160836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.287034,0.0,0.0,0.000000,0.0
9506,163937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [88]:
_

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [87]:
df

,userId,movieId,rating,timestamp,u_1,u_2,u_3,u_4,u_5,u_6,...,i_291,i_292,i_293,i_294,i_295,i_296,i_297,i_298,i_299,i_300
94613,599,52328,4.0,1498499032,446,163,166,975,250,367,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.448248,0.0,0.0,0.0
43264,290,968,4.0,975032333,35,7,11,64,12,48,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
47294,307,4876,2.0,1186172830,153,45,94,518,107,181,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
56885,380,19,5.0,1493667522,391,151,163,370,219,88,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
83899,534,70305,3.5,1459796557,253,84,83,216,112,56,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38278,263,988,3.0,941590705,49,20,22,98,20,88,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
30685,215,3147,4.0,1260909340,16,0,1,42,9,27,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
60644,391,2598,1.0,1030825353,70,13,16,122,25,82,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
5564,40,48,2.0,832059371,23,8,18,41,12,22,...,0.0,0.0,0.165496,0.0,0.0,0.141723,0.000000,0.0,0.0,0.0


In [52]:
_, user_df = create_user_feature()
item_df = create_item_feature(num_features = 300)
df = get_augmented_table()
df = df.fillna(0)


df = df.sample(500)

item_ids = item_df['movieId'].unique()
user_ids = df['userId'].unique()

In [53]:
# # df, item_df, user_df, item_ids, user_ids = load_data('sample_data/augmented_transaction_table.parquet',
# #                                                     'sample_data/item_feature.parquet',
# #                                                     'sample_data/user_feature.parquet')


# df = df
# item_df = item_df
# user_df = user_df
# item_ids = item_ids
# user_ids = user_ids

## Fit model

In [75]:
#Declare your model
rs_model1 = RandomForestRegressor(random_state=202109, n_jobs=-1)

#Do your train and test split
df_train, df_test, df_test_um, indx_train, indx_test = split_train_test(df, 0.7) #To split the data

# df_train = df_train.sample(100)

#Fit your model to the train data
model_fit = fit_ml_cb(df_train, rs_model1) #To fit the model

#Predict on the test data
preds_array = reco_ml_cb_tt(df_test, model_fit) #To make predictions as an array

In [76]:
preds_array

array([3.43 , 3.375, 3.155, 3.69 , 3.785, 3.625, 2.445, 3.045, 3.27 ,
       3.76 , 3.68 , 3.315, 4.175, 3.765, 3.43 , 3.395, 3.355, 3.23 ,
       3.645, 3.495, 3.625, 3.385, 3.3  , 3.84 , 3.185, 3.82 , 3.75 ,
       3.41 , 3.825, 3.235, 3.68 , 2.665, 2.64 , 2.355, 3.64 , 3.675,
       3.37 , 3.615, 3.755, 3.75 , 2.715, 4.045, 4.15 , 3.175, 3.36 ,
       2.885, 3.65 , 3.625, 3.385, 3.585, 3.745, 3.42 , 3.205, 3.05 ,
       3.745, 3.36 , 3.375, 3.37 , 3.755, 3.7  , 3.46 , 3.225, 3.33 ,
       3.385, 3.775, 3.655, 3.555, 3.55 , 2.89 , 3.655, 3.56 , 3.64 ,
       3.575, 3.72 , 3.485, 3.475, 3.79 , 3.575, 2.84 , 3.675, 3.77 ,
       3.71 , 3.46 , 4.08 , 4.03 , 3.185, 3.205, 3.185, 3.795, 3.02 ,
       3.5  , 3.585, 3.46 , 3.255, 4.125, 3.975, 3.145, 3.045, 3.195,
       3.335, 3.65 , 3.915, 2.015, 3.51 , 3.695, 3.805, 3.52 , 3.005,
       3.33 , 3.265, 3.165, 3.52 , 3.635, 2.905, 2.61 , 2.53 , 3.78 ,
       3.035, 3.23 , 3.615, 3.625, 2.52 , 4.435, 3.275, 2.96 , 3.385,
       3.735, 3.33 ,

## Predict

In [56]:
from resype import *

In [57]:
def get_rec(utility_matrix_preds, utility_matrix, user_list, top_n, uc_assignment=None):
    
    """Returns the top N item cluster recommendations for each user in the user list
    
            Parameters:
                    utility_matrix (numpy.ndarray): Matrix of utilities for each user-item pairing
                    utility_matrix_o (numpy.ndarray): Original utility matrix, before imputation
                    user_list (array-like): List of users
                    uc_assignment (array-like): List containing the cluster assignment of each user
                    top_n (int): Number of item clusters to recommend

            Returns:
                    df_rec (pandas.DataFrame): Table containing the top N item cluster recommendations for each user in the user list
                    
    """
    
    # Class stuff
    utility_matrix_o = utility_matrix.fillna(0).values
    utility_matrix = utility_matrix_preds.values
    
    # Don't recommend items that are already rated
    utility_matrix[np.where(utility_matrix_o != 0)] = -np.inf
    
    # Get top N per user cluster
    cluster_rec = utility_matrix.argsort()[:, -top_n:]

    # Create recommendation table
    df_rec = pd.DataFrame()
    df_rec['user_id'] = user_list
                
    for i in range(top_n):
        df_rec['rank_'+str(i+1)] = np.zeros(df_rec.shape[0])
        for j in range(df_rec.shape[0]):
            if uc_assignment is None:
                df_rec.iloc[j, i+1] = cluster_rec[user_list[j], top_n-i-1]
            else:
                df_rec.iloc[j, i+1] = cluster_rec[uc_assignment.iloc[user_list[j], 0], top_n-i-1]
    
    # look-up tables
    if uc_assignment is None:
        user_id_lookup = utility_matrix_preds.index
        item_id_lookup = utility_matrix_preds.columns
        for j in range(df_rec.shape[0]):
            df_rec.iloc[j, 0] = user_id_lookup[df_rec.iloc[j, 0].astype('int32')]
            for i in range(top_n):
                df_rec.iloc[j, i+1] = item_id_lookup[df_rec.iloc[j, i+1].astype('int32')]
    
    return df_rec

In [58]:
from resype.resype import Resype

In [59]:
user_ = user_df.drop(['userId'], axis=1)

In [60]:
user_ = user_.sample(10)

In [61]:
item_ = item_df.drop(['movieId'], axis=1)

In [62]:
reco_mtx = reco_ml_cb(user_, item_, item_ids, model_fit)
reco_mtx

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
346,3.480,3.200,3.345,3.450,3.365,3.075,3.580,3.510,3.470,3.110,...,3.405,3.630,3.290,3.310,3.420,3.350,3.560,3.480,3.285,3.560
216,3.610,3.580,3.495,3.625,3.595,3.135,3.700,3.450,3.725,3.445,...,3.465,3.660,3.425,3.520,3.335,3.575,3.700,3.505,3.400,3.695
148,3.395,3.275,3.380,3.320,3.365,3.085,3.465,3.415,3.415,3.230,...,3.275,3.525,3.310,3.285,3.320,3.170,3.485,3.335,3.310,3.475
82,3.720,3.375,3.545,3.740,3.625,3.190,3.895,3.600,3.705,3.340,...,3.640,3.860,3.545,3.585,3.515,3.610,3.805,3.650,3.515,3.800
43,3.070,2.985,3.070,3.030,3.010,2.815,3.245,3.110,3.060,2.910,...,3.015,3.215,2.995,2.985,3.035,2.915,3.155,3.045,3.015,3.145
586,3.400,3.240,3.390,3.300,3.220,2.930,3.445,3.390,3.435,3.085,...,3.135,3.500,3.195,3.290,3.155,3.150,3.445,3.310,3.300,3.465
347,3.785,3.505,3.640,3.745,3.620,3.285,3.815,3.765,3.780,3.370,...,3.580,3.880,3.530,3.695,3.565,3.530,3.865,3.690,3.545,3.845
90,3.995,3.810,3.845,4.045,3.910,3.400,4.005,3.780,4.025,3.730,...,3.870,4.035,3.740,3.825,3.605,3.930,4.065,3.835,3.660,4.075
519,3.965,3.685,3.805,3.955,3.795,3.305,3.985,3.845,3.955,3.680,...,3.640,4.040,3.790,3.915,3.585,3.730,4.005,3.770,3.890,4.035
221,3.710,3.395,3.475,3.795,3.590,3.035,3.870,3.560,3.765,3.385,...,3.480,3.800,3.470,3.535,3.435,3.660,3.785,3.615,3.450,3.765


In [63]:
reco_inds = np.array(reco_mtx.index)
reco_inds

array([346, 216, 148,  82,  43, 586, 347,  90, 519, 221])

In [64]:
um = _.drop('timestamp', axis=1)
um.columns = ['user_id', 'item_id', 'rating']

rs = Resype(um)
original_um = rs.construct_utility_matrix()
original_um = original_um.loc[reco_inds[:], :]
original_um = original_um[reco_mtx.columns]

In [65]:
reco_mtx.shape, original_um.shape

((10, 9508), (10, 9508))

In [66]:
original_um = original_um.fillna(0)
original_um

item_id,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
user_id,,,,,,,,,,,,,,,,,,,,,
346,0.0,0.0,0.0,3.5,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
216,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,2.5,0.0,3.5,3.5,0.0,0.0,0.0,4.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,5.0,5.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
347,5.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
reco_mtx = reco_mtx.fillna(0)
reco_mtx

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
346,3.480,3.200,3.345,3.450,3.365,3.075,3.580,3.510,3.470,3.110,...,3.405,3.630,3.290,3.310,3.420,3.350,3.560,3.480,3.285,3.560
216,3.610,3.580,3.495,3.625,3.595,3.135,3.700,3.450,3.725,3.445,...,3.465,3.660,3.425,3.520,3.335,3.575,3.700,3.505,3.400,3.695
148,3.395,3.275,3.380,3.320,3.365,3.085,3.465,3.415,3.415,3.230,...,3.275,3.525,3.310,3.285,3.320,3.170,3.485,3.335,3.310,3.475
82,3.720,3.375,3.545,3.740,3.625,3.190,3.895,3.600,3.705,3.340,...,3.640,3.860,3.545,3.585,3.515,3.610,3.805,3.650,3.515,3.800
43,3.070,2.985,3.070,3.030,3.010,2.815,3.245,3.110,3.060,2.910,...,3.015,3.215,2.995,2.985,3.035,2.915,3.155,3.045,3.015,3.145
586,3.400,3.240,3.390,3.300,3.220,2.930,3.445,3.390,3.435,3.085,...,3.135,3.500,3.195,3.290,3.155,3.150,3.445,3.310,3.300,3.465
347,3.785,3.505,3.640,3.745,3.620,3.285,3.815,3.765,3.780,3.370,...,3.580,3.880,3.530,3.695,3.565,3.530,3.865,3.690,3.545,3.845
90,3.995,3.810,3.845,4.045,3.910,3.400,4.005,3.780,4.025,3.730,...,3.870,4.035,3.740,3.825,3.605,3.930,4.065,3.835,3.660,4.075
519,3.965,3.685,3.805,3.955,3.795,3.305,3.985,3.845,3.955,3.680,...,3.640,4.040,3.790,3.915,3.585,3.730,4.005,3.770,3.890,4.035
221,3.710,3.395,3.475,3.795,3.590,3.035,3.870,3.560,3.765,3.385,...,3.480,3.800,3.470,3.535,3.435,3.660,3.785,3.615,3.450,3.765


## Get recommendations

In [68]:

get_rec(reco_mtx, original_um, user_list = [0, 2], top_n = 3)

,user_id,rank_1,rank_2,rank_3
0,346,237.0,4971.0,581.0
1,148,237.0,3161.0,650.0


## Evaluate

In [69]:
evaluate_arrays(preds_array, df, indx_test) #MSE and MAE

proceed


(1.0785096899224806, 0.8201550387596899)

## Cross Validation

In [70]:
df

,userId,movieId,rating,timestamp,u_1,u_2,u_3,u_4,u_5,u_6,...,i_291,i_292,i_293,i_294,i_295,i_296,i_297,i_298,i_299,i_300
94613,599,52328,4.0,1498499032,446,163,166,975,250,367,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.448248,0.0,0.0,0.0
43264,290,968,4.0,975032333,35,7,11,64,12,48,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
47294,307,4876,2.0,1186172830,153,45,94,518,107,181,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
56885,380,19,5.0,1493667522,391,151,163,370,219,88,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
83899,534,70305,3.5,1459796557,253,84,83,216,112,56,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38278,263,988,3.0,941590705,49,20,22,98,20,88,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
30685,215,3147,4.0,1260909340,16,0,1,42,9,27,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
60644,391,2598,1.0,1030825353,70,13,16,122,25,82,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
5564,40,48,2.0,832059371,23,8,18,41,12,22,...,0.0,0.0,0.165496,0.0,0.0,0.141723,0.000000,0.0,0.0,0.0


In [80]:
mse, mae = cross_val(df, 5, rs_model1, split_method='random')

0
Starting splitting
Finished splitting
Starting training
Finished training
Starting completing matrix
Finished completing matrix
Starting computing MAE and MSE
proceed
Finished computing MAE and MSE
1
Starting splitting
Finished splitting
Starting training
Finished training
Starting completing matrix
Finished completing matrix
Starting computing MAE and MSE
proceed
Finished computing MAE and MSE
2
Starting splitting
Finished splitting
Starting training
Finished training
Starting completing matrix
Finished completing matrix
Starting computing MAE and MSE
proceed
Finished computing MAE and MSE
3
Starting splitting
Finished splitting
Starting training
Finished training
Starting completing matrix
Finished completing matrix
Starting computing MAE and MSE
proceed
Finished computing MAE and MSE
4
Starting splitting
Finished splitting
Starting training
Finished training
Starting completing matrix
Finished completing matrix
Starting computing MAE and MSE
proceed
Finished computing MAE and MSE


In [81]:
mse, mae

([1.223570930232558,
  1.1374905038759688,
  1.147925968992248,
  1.1346201550387598,
  1.3550001937984497],
 [0.8675968992248062,
  0.8425581395348838,
  0.8515116279069768,
  0.8392635658914729,
  0.9177906976744187])